# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import os
import cv2
import sys
sys.path.append("/usr/local/lib")
import time
import torch
import IPython
import traitlets
import threading
import torchvision
import numpy as np
import pyrealsense2 as rs
import ipywidgets.widgets as widgets
from uuid import uuid1
from IPython.display import display
import torch.nn.functional as F


In [2]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [3]:
model.load_state_dict(torch.load('best_model_green_100.pth'))

In [4]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [5]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [6]:
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

display(widgets.HBox([image, blocked_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [7]:
from jetbot import Robot

robot = Robot(driver_board = "waveshare")

In [8]:
prob_blocked = 0
def rs_on():
    pipeline = rs.pipeline()
    config = rs.config()
    #config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 6)
    pipeline.start(config)

    try:
        while True:
            global prob_blocked 
            frames = pipeline.wait_for_frames()
            #depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not color_frame:
                continue

            # Convert images to numpy arrays
            #depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            new_color_image = cv2.resize(color_image, (224, 224), interpolation=cv2.INTER_CUBIC)
            x = preprocess(new_color_image)
            y = model(x)
            y = F.softmax(y, dim=1)
            prob_blocked = float(y.flatten()[0])
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
           #  depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

             # Stack both images horizontally
            #images = np.hstack((color_image, depth_colormap))

            # Show images
            image.value = cv2.imencode('.jpg',new_color_image)[1].tobytes()
            
    finally:
        # Stop streaming
        pipeline.stop()
        robot.stop()

thread =threading.Thread(target=rs_on)
thread.start()


In [ ]:
while True:
    global blocked_slider, robot, prob_blocked
    
    blocked_slider.value = prob_blocked
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    time.sleep(0.001)